#### Project -5 Recommender System in Spark
#### Author :Santosh Manjrekar

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
import numpy as np
import math 
import time


In [10]:
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load data from the path to a dataframe called "ratings"
## Small Dataset
##rating = spark.read.format("csv").option("header", "true").load("hdfs://saltdean/data/movielens/ml-latest-small/ratings.csv")
## Large Dataset
rating = spark.read.format("csv").option("delimiter", "\t").option("header", "true").load("ratings.csv")
# Check which features are present 
print("rating")
rating.show()

rating
+---+-------+--------+------+---------+-----------+------------+
|_c0|user_id|movie_id|rating|timestamp|user_emb_id|movie_emb_id|
+---+-------+--------+------+---------+-----------+------------+
|  0|      1|    1193|     5|978300760|          0|        1192|
|  1|      1|     661|     3|978302109|          0|         660|
|  2|      1|     914|     3|978301968|          0|         913|
|  3|      1|    3408|     4|978300275|          0|        3407|
|  4|      1|    2355|     5|978824291|          0|        2354|
|  5|      1|    1197|     3|978302268|          0|        1196|
|  6|      1|    1287|     5|978302039|          0|        1286|
|  7|      1|    2804|     5|978300719|          0|        2803|
|  8|      1|     594|     4|978302268|          0|         593|
|  9|      1|     919|     4|978301368|          0|         918|
| 10|      1|     595|     5|978824268|          0|         594|
| 11|      1|     938|     4|978301752|          0|         937|
| 12|      1|    2

#### Split Training and Testing data in 80/20 ratio with the option of further reducing training set size

In [11]:
## Change the Dataset size

#(rating, garbage) = rating.randomSplit([0.999, 0.001]) # ~100,000 rows
# (rating, garbage) = rating.randomSplit([0.1, 0.9])       # ~10,000 rows
#(rating, garbage) = rating.randomSplit([0.01, 0.99])   # ~1000 rows
# (rating, garbage) = rating.randomSplit([0.001, 0.999]) # ~100 rows
# Print dataset size
print('dataset data size: ', rating.count(),' rows') 
# --------------------------------------------------------------------------- #
# Split the data into training (80%) and hold-out testing data (20%)
(training, test) = rating.randomSplit([0.8, 0.2])
# --------------------------------------------------------------------------- #
# Reduce the training data set (*uncomment all to keep with the original size) 
#(training, garbage) = training.randomSplit([0.8, 0.2])   # ~50%
# (training, garbage) = training.randomSplit([0.25, 0.75]) # ~25% 
# (training, garbage) = training.randomSplit([0.1, 0.9])   # ~10% 
# (training, garbage) = training.randomSplit([0.01, 0.99]) # ~1% 
# --------------------------------------------------------------------------- #
# Print traindata size
print('training data size: ',training.count(),' rows') 
# Print training data size
print('test data size: ',test.count(),' rows')

dataset data size:  1000209  rows
training data size:  800317  rows
test data size:  199892  rows


In [13]:
## Manipulation of the Data

# Load data from path to dataframe called "movies" (all the movies)
movies = spark.read.format("csv").option("delimiter", "\t").option("header", "true").load("movies.csv")
# Check which features are present 
print("movies")
movies.show()

# Join the columns of "movies" with "training" and "test" datasets into "movielens_training" and "movielens_test"
movielens_training = training.join(movies, "movie_id")
movielens_test = test.join(movies, "movie_id")

# Cast data type from String to Integer and Double for training and test datasets
movielens_training = movielens_training.withColumn("movie_id", training["movie_id"].cast(IntegerType()))
movielens_training = movielens_training.withColumn("rating", training["rating"].cast(DoubleType()))
movielens_training = movielens_training.withColumn("timestamp", training["timestamp"].cast(IntegerType()))
movielens_training = movielens_training.withColumn("user_id", training["user_id"].cast(IntegerType()))

movielens_test = movielens_test.withColumn("movie_id", test["movie_id"].cast(IntegerType()))
movielens_test = movielens_test.withColumn("rating", test["rating"].cast(DoubleType()))
movielens_test = movielens_test.withColumn("timestamp", test["timestamp"].cast(IntegerType()))
movielens_test = movielens_test.withColumn("user_id", test["user_id"].cast(IntegerType()))

# ----- Print the training -----
print("\n Training")
training_count = movielens_training.count()
print('movielens_training data size: ',training_count,' rows') 
movielens_training.show(50)
# Print the types used in each column
print("Type of each Column \n")
movielens_training.printSchema()

# ----- Print the test -----
print("\n Test")
test_count=movielens_test.count()
print('movielens_test data size: ',test_count,' rows') 
movielens_test.show(50)
# Print the types used in each column
print("Type of each Column \n")
movielens_test.printSchema()

# Create the whole dataset again with less data
print("\n All Dataset")
all_count = test_count+training_count
print('All Dataset size: ',(all_count),' rows') 
movie_ratings = movielens_training.unionAll(movielens_test)
movie_ratings.show()


movies
+---+--------+--------------------+--------------------+
|_c0|movie_id|               title|              genres|
+---+--------+--------------------+--------------------+
|  0|       1|    Toy Story (1995)|Animation|Childre...|
|  1|       2|      Jumanji (1995)|Adventure|Childre...|
|  2|       3|Grumpier Old Men ...|      Comedy|Romance|
|  3|       4|Waiting to Exhale...|        Comedy|Drama|
|  4|       5|Father of the Bri...|              Comedy|
|  5|       6|         Heat (1995)|Action|Crime|Thri...|
|  6|       7|      Sabrina (1995)|      Comedy|Romance|
|  7|       8| Tom and Huck (1995)|Adventure|Children's|
|  8|       9| Sudden Death (1995)|              Action|
|  9|      10|    GoldenEye (1995)|Action|Adventure|...|
| 10|      11|American Presiden...|Comedy|Drama|Romance|
| 11|      12|Dracula: Dead and...|       Comedy|Horror|
| 12|      13|        Balto (1995)|Animation|Children's|
| 13|      14|        Nixon (1995)|               Drama|
| 14|      15|Cutthroat 

+--------+------+-------+------+---------+-----------+------------+----+--------------------+--------------------+
|movie_id|   _c0|user_id|rating|timestamp|user_emb_id|movie_emb_id| _c0|               title|              genres|
+--------+------+-------+------+---------+-----------+------------+----+--------------------+--------------------+
|     661|     1|      1|   3.0|978302109|          0|         660| 655|James and the Gia...|Animation|Childre...|
|     595|    10|      1|   5.0|978824268|          0|         594| 591|Beauty and the Be...|Animation|Childre...|
|    3654|   100|      2|   3.0|978298814|          1|        3653|3585|Guns of Navarone,...|    Action|Drama|War|
|    2762|  1000|     10|   5.0|978225541|          9|        2761|2693|Sixth Sense, The ...|            Thriller|
|    3175| 10000|     70|   5.0|977877962|         69|        3174|3106| Galaxy Quest (1999)|Adventure|Comedy|...|
|       1|100000|    669|   5.0|975631430|        668|           0|   0|    Toy 

In [14]:
## Create the recommender system 

print("Create Training Algorithm (ALS)")
# Create an Alternate Least Square learning algorithm (Estimator)
als = ALS(rank=10, 
          maxIter=10,
          userCol="user_id",   
          itemCol="movie_id",  
          ratingCol="rating")
### numBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).
### rank is the number of latent factors in the model (defaults to 10).
### maxIter is the maximum number of iterations to run (defaults to 10).
### regParam specifies the regularization parameter in ALS (defaults to 1.0).
print("Create Parameter Grid Builder")
## Create a ParamGridBuilder to construct a grid of parameters to search over. (ParameterMaps)
paramGrid = ParamGridBuilder().addGrid(als.rank, [5,10,20,30])\
                                 .addGrid(als.maxIter, [5,10,20,30])\
                                 .build()
            
## No Grid Search
#paramGrid = ParamGridBuilder().build() # * UnComment this line and comment the block above to run quickly

# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# In this case the estimator is simply the linear regression. (Evaluator)
regEval = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

#### ------------------------ TRAINING ------------------------ ####
# Get start time
s=time.time() # training time

# Train and Validate models
tvs = TrainValidationSplit(estimator=als,
                            estimatorParamMaps=paramGrid,
                            evaluator=regEval,
                            # 80% of the data will be used for training, 20% for validation.
                            trainRatio=0.8, seed=5)

# Run TrainValidationSplit to train and choose the best set of parameters.
model = tvs.fit(movielens_training)

# Get the best model parameters
best_model = model.bestModel
maxIter = (best_model
    ._java_obj     # Get Java object
    .parent()      # Get parent (ALS estimator)
    .getMaxIter()) # Get best maxIter

rank = best_model.rank # Get best rank
print("rank:", rank)
print("maxIter:", maxIter)
print('\n-----------')

# Get end time
e=time.time() # training time

# Test the model's prediction in the hold-out Training data           
predictions = best_model.transform(movielens_training)

# Drop any rows with nan values from prediction (due to cold start problem)
# predictions = predictions.dropna()
predictions = predictions.fillna(0);

# Evaluate the overall performance of the model by computing the Root-mean-square error (RMSE) on the Training data
rmse = regEval.evaluate(predictions)
print("Training Error (RMS) = ", round(rmse, 4))

## Print the size of training data
# print('Training data size: ',training.count(),' rows')  
     
# Print the time spent to train
print('Training time: ',round(e-s, 4),' seconds')

#### ------------------------ TESTING ------------------------ ####
# Get relative time
s=time.time() # testing time

# Test the model's prediction in the hold-out Test data           
predictions = best_model.transform(movielens_test)

# Get end time
e=time.time() # testing time

# Drop or replace with 0 any rows with nan values from prediction (due to cold start problem)
predictions = predictions.dropna()
# predictions = predictions.fillna(0);
               
# Evaluate the overall performance of the model by computing the Root-mean-square error (RMSE) on the Test data
rmse = regEval.evaluate(predictions)
print('')
print("Test Error (RMS) = ", round(rmse, 4))

## Print the size of test data
# print('Test data size: ',test.count(),' rows')  
     
# Print the time spent to test
print('Test time: ',round(e-s, 4),' seconds')

####------ Results for training and testing without Grid Search ------------------####
## using default parameters for rank (10) and number of iterations(10) 

## Training data size: ~80,000 rows
# Training Error (RMS) =  0.6382  || Test Error (RMS) =  1.1182
# Training time:  10.497  seconds || Test time:  0.0601  seconds

## Training data size: ~8,000 rows
# Training Error (RMS) =  0.2326  || Test Error (RMS) =  2.8208
# Training time:  7.6864  seconds || Test time:  0.0603  seconds
    
## Training data size: ~800 rows
# Training Error (RMS) =   0.0547 || Test Error (RMS) =  3.7398
# Training time:  6.492  seconds  || Test time:  0.0514  seconds
 
## Training data size: ~80 rows
# Training Error (RMS) =  0.0501  || Test Error (RMS) =  3.7025
# Training time:  5.419  seconds  || Test time:  0.0504  seconds

# As we can see the larger the dataset the more computational expensive it is. 
# The training error is low and testing error is high for small dataset (meaning overfitting). 
# When the dataset is large, the model generalizes better, reducing overfitting.
# One important point is that, only when the data set is really big (around 80,000) 
# is when the model starts to having a good prediction

# The Parameter Grid Search was applied twice on two different dataset sizes - 100,000 rows and 20 M rows
# and the results are documented below:. 

# ----------- Smaller Dataset Param Grid --------------------------

#Dataset size 100,000
# 80:20 training:test split 
#best parameters: rank: 5, maxIter: 5
# Training Error (RMS) =  0.6765    || Test Error (RMS) =  0.9161
# Training time:  276.0033  seconds || Test time:  0.0602  seconds


#-------- Largest Dataset Param Grid ----------------------

# dataset data size:  19980310  rows
#training data size:  15986149  rows
#test data size:  3994161  rows
#best parameters: rank: 5, maxIter: 5
#Training Error (RMS) =  0.7965        ||Test Error (RMS) =  0.8203
#Training time:  3296.2835  seconds    ||Test time:  0.0774  seconds


Create Training Algorithm (ALS)
Create Parameter Grid Builder


Py4JJavaError: An error occurred while calling o863.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.io.ObjectOutputStream.writeObject0(Unknown Source)
	at java.io.ObjectOutputStream.defaultWriteFields(Unknown Source)
	at java.io.ObjectOutputStream.writeSerialData(Unknown Source)
	at java.io.ObjectOutputStream.writeOrdinaryObject(Unknown Source)
	at java.io.ObjectOutputStream.writeObject0(Unknown Source)
	at java.io.ObjectOutputStream.defaultWriteFields(Unknown Source)
	at java.io.ObjectOutputStream.writeSerialData(Unknown Source)
	at java.io.ObjectOutputStream.writeOrdinaryObject(Unknown Source)
	at java.io.ObjectOutputStream.writeObject0(Unknown Source)
	at java.io.ObjectOutputStream.defaultWriteFields(Unknown Source)
	at java.io.ObjectOutputStream.writeSerialData(Unknown Source)
	at java.io.ObjectOutputStream.writeOrdinaryObject(Unknown Source)
	at java